# Creation of a digital twin 
This is a KWH application which allows creating a simple digital twin with a service using python reference implementation. All necessary modules are imported into the script, including the S³I library and reference implementation library. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64 
import getpass
import requests
import collections
from threading import Thread, Event
import sys
from tools import print_with_timestamp, check_message_encryption, yes, no## Configure the notebook

## Configure the notebook
In order to use the S³I this notebook needs a client id and the respective secret. Those are already prepared for you. You need to enter your username and password in the following input fields in order to assign this notebook to your personal KWH application. 

In [ ]:
print_with_timestamp("KWH application to create a dt, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=dt_creation_app_id,
                                             client_secret=dt_creation_app_secret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token'''
parsed_username = jwt.decode(access_token, verify=False)["preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

## Create an identity for your digital twin
your digital twin needs also an identify, which can be created using config rest api. 